# Using pymatgen

In [1]:
import pandas as pd
import numpy as np
from mcbac import CifHelper
import mcbac

In [2]:
import pymatgen
from pymatgen.io.cif import CifParser

In [3]:
# load cif and get dict
p = CifParser('./Example.cif')
d = p.as_dict()

In [4]:
# make helper object
c = CifHelper(d['NiH16C24N2O5'])

In [13]:
c.nebr_stack_charge()

,nebr_0,nebr_1,nebr_2,val_0,val_1,val_2,val_total,val_shift,val_zero
0,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
1,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
2,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
3,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
4,H,C,CHN,0.124004,0.102895,0.086959,0.086959,-0.001120,0.088079
...,...,...,...,...,...,...,...,...,...
187,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
188,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
189,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
190,O,CC,CCCCOO,-0.641530,-0.255959,-0.235748,-0.235748,-0.003035,-0.232713


## from gemmi

In [14]:
import gemmi.cif
import json

In [18]:
gemmi.cif.read_file

<instancemethod as_json at 0x7fca80a30970>

In [15]:
# note that the File Example.cif has a weird header
# this fixes that
with open('./Example.cif','r') as f:
    out = f.readlines()

with open('./Example.gemmi.cif', 'w') as f:
    f.writelines(out[1:])

In [16]:
# or using gemmi
g = gemmi.cif.read_file('./Example.gemmi.cif')

d_gemmi = json.loads(g.as_json(lowercase_names=False))
c2 = CifHelper(d_gemmi['NiH16C24N2O5'])

In [17]:
g

<gemmi.cif.Document with 1 blocks (NiH16C24N2O5)>

In [9]:
pd.testing.assert_frame_equal(c.nebr_stack(), c2.nebr_stack())

## from methods

In [10]:
dict_of_objects_0 = CifHelper.from_pymatgen('./Example.cif', sole_block=False )

dc0 = dict_of_objects_0['NiH16C24N2O5']

dict_of_objects_1 = CifHelper.from_gemmi('./Example.gemmi.cif', sole_block=False)

dc1 = dict_of_objects_1['nih16c24n2o5']

In [11]:
pd.testing.assert_frame_equal(c.nebr_stack(), dc0.nebr_stack())
pd.testing.assert_frame_equal(c.nebr_stack(), dc1.nebr_stack())

In [12]:
# or just use sole_block=True, which is the default

In [13]:
dc0 = CifHelper.from_pymatgen('./Example.cif')
dc1 = CifHelper.from_gemmi('./Example.gemmi.cif',)

In [14]:
pd.testing.assert_frame_equal(c.nebr_stack(), dc0.nebr_stack())
pd.testing.assert_frame_equal(c.nebr_stack(), dc1.nebr_stack())

## Basic functionality

In [15]:
# directly access things from cif file
c['_cell_length_a']

'11.26900037'

In [16]:
# created renamers for easy access and float/array conversion 
# see class definition for more info
print(c.a)

11.26900037


In [17]:
help(c)

Help on CifHelper in module mcbac.core object:

class CifHelper(builtins.object)
 |  CifHelper(data, radii_dict=None, name=None)
 |  
 |  Helper class to work with cif file
 |  
 |  Paramters
 |  ---------
 |  data : dict
 |      dictionary from cif file
 |  radii_dict : dict, optional
 |      mapping from elements -> radii
 |  
 |  Attributes
 |  ----------
 |  a, b, c : float
 |      values from data['_cell_length_a'] etc
 |  alpha, beta, gamma : float
 |      value from data['_cell_angle_alpha], etc
 |  label : array of strings
 |      value of data['_atom_site_label']
 |  type_sybol : array of strings
 |      value from data['_atom_site_type_symbol']
 |      or data['_atom_site_label] if type_symbol not available.
 |  frac_x, frac_y, frac_z, charge : array of floats
 |      value from data['_atom_site_fract_x'], etc
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, index)
 |  
 |  __init__(self, data, radii_dict=None, name=None)
 |      Initialize self.  See help(type(self)

### cartisian coords

In [18]:
# coordinates -> c.coords
# replicated coords -> c.coords_replicated(stack=?)

### distance calc

In [19]:
# unstacked has shape (n, 27, n)
# first dim is for coords, second for replicate id, third replicate atom id
# if stacked, then shape = (n, 27 * n)
c.distance_matrix(stack=True)

array([[31.49257799, 27.47389044, 21.95667209, ..., 41.29246609,
        46.47352217, 40.24557049],
       [36.51919306, 31.49257799, 23.96030471, ..., 36.38409112,
        41.58302591, 34.37499822],
       [46.03417839, 40.40185054, 31.49257799, ..., 28.56033942,
        33.75433917, 25.25947674],
       ...,
       [41.29246609, 36.38409112, 28.56033942, ..., 31.49257799,
        36.91356271, 29.89882206],
       [46.47352217, 41.58302591, 33.75433917, ..., 26.21199769,
        31.49257799, 24.65199383],
       [40.24557049, 34.37499822, 25.25947674, ..., 34.83530307,
        40.03525782, 31.49257799]])

### distance dataframe

In [20]:
c.distance_frame

,cntr,repl,nebr,cntr_radii,nebr_radii,cntr_name,nebr_name,distance
0,0,0,0,1.164,1.164,Ni,Ni,31.492578
1,0,0,1,1.164,1.164,Ni,Ni,27.473890
2,0,0,2,1.164,1.164,Ni,Ni,21.956672
3,0,0,3,1.164,1.164,Ni,Ni,23.960305
4,0,0,4,1.164,0.354,Ni,H,16.968970
...,...,...,...,...,...,...,...,...
995323,191,26,187,0.634,0.634,O,O,31.157935
995324,191,26,188,0.634,0.634,O,O,35.151915
995325,191,26,189,0.634,0.634,O,O,34.835303
995326,191,26,190,0.634,0.634,O,O,40.035258


### nearest neighbor frame

In [21]:
c.nebr_frame(rcut_fac=1.25)

,cntr,repl,nebr,cntr_radii,nebr_radii,cntr_name,nebr_name,distance,rcut,rank
0,0,13,0,1.164,1.164,Ni,Ni,0.000000e+00,2.91000,0
1,0,13,174,1.164,0.634,Ni,O,2.008001e+00,2.24750,1
2,0,12,185,1.164,0.634,Ni,O,2.036361e+00,2.24750,2
3,0,13,165,1.164,0.700,Ni,N,2.087743e+00,2.33000,3
4,0,13,164,1.164,0.700,Ni,N,2.098025e+00,2.33000,4
...,...,...,...,...,...,...,...,...,...,...
627,190,13,140,0.634,0.757,O,C,1.385512e+00,1.73875,1
628,190,13,143,0.634,0.757,O,C,1.390848e+00,1.73875,2
629,191,13,191,0.634,0.634,O,O,8.881784e-16,1.58500,0
630,191,13,157,0.634,0.757,O,C,1.386235e+00,1.73875,1


### nebr of nebr frame

In [22]:
# second neighbors of 'cntr' are in 'nebr_2nd'
c.nebr_nebr_frame(rcut_fac=1.25)

,cntr,repl,nebr,cntr_radii,nebr_radii,cntr_name,nebr_name,distance,rcut,rank,cntr_2nd,repl_2nd,nebr_2nd,cntr_radii_2nd,nebr_radii_2nd,cntr_name_2nd,nebr_name_2nd,distance_2nd,rcut_2nd,rank_2nd
0,0,13,174,1.164,0.634,Ni,O,2.008001,2.24750,1,174,13,102,0.634,0.757,O,C,1.245089,1.73875,1
1,0,13,174,1.164,0.634,Ni,O,2.008001,2.24750,1,174,13,0,0.634,1.164,O,Ni,2.008001,2.24750,2
2,0,12,185,1.164,0.634,Ni,O,2.036361,2.24750,2,185,13,150,0.634,0.757,O,C,1.261929,1.73875,1
3,0,12,185,1.164,0.634,Ni,O,2.036361,2.24750,2,185,14,0,0.634,1.164,O,Ni,2.036361,2.24750,2
4,0,13,165,1.164,0.700,Ni,N,2.087743,2.33000,3,165,13,75,0.700,0.757,N,C,1.324517,1.82125,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,191,13,157,0.634,0.757,O,C,1.386235,1.73875,1,157,13,158,0.757,0.757,C,C,1.384673,1.89250,2
1220,191,13,157,0.634,0.757,O,C,1.386235,1.73875,1,157,13,191,0.757,0.634,C,O,1.386235,1.73875,3
1221,191,13,154,0.634,0.757,O,C,1.395728,1.73875,2,154,4,155,0.757,0.757,C,C,1.358932,1.89250,1
1222,191,13,154,0.634,0.757,O,C,1.395728,1.73875,2,154,13,153,0.757,0.757,C,C,1.364052,1.89250,2


### 0th, 1st, and 2nd neighbor keys

In [23]:
c.nebr_stack()

,nebr_0,nebr_1,nebr_2
cntr,,,
0,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi
1,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi
2,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi
3,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi
4,H,C,CHN
...,...,...,...
187,O,CNi,CNNOOOOOO
188,O,CNi,CNNOOOOOO
189,O,CNi,CNNOOOOOO


In [24]:
# test this is the same as from other code
other = pd.read_csv('./Example.cif.dat.total.txt', header=None, sep='\s+').values
np.testing.assert_array_equal(c.nebr_stack().values, other)

## Thats where I'm at.  Not sure what to do with the databases

I don't understand what the shell script is trying to do.  The first read merge I get, but the other ones make no sense.

In [25]:
# OK, I now realize what the awk code is doing
# First tries to use triples, then doubles, then singles (2nd, 1st, 0th)

In [26]:
out = c.nebr_stack_charge()

In [27]:
out

,nebr_0,nebr_1,nebr_2,val_0,val_1,val_2,val_total,val_shift,val_zero
0,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
1,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
2,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
3,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
4,H,C,CHN,0.124004,0.102895,0.086959,0.086959,-0.001120,0.088079
...,...,...,...,...,...,...,...,...,...
187,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
188,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
189,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
190,O,CC,CCCCOO,-0.641530,-0.255959,-0.235748,-0.235748,-0.003035,-0.232713


In [28]:
# test against awk values...
other_total = pd.read_csv('./Final_Charge_Example.cif.txt',header=None)[0].values
other_shift = pd.read_csv('./Shift_Charge_Example.cif.txt',header=None)[0].values
other_zero = pd.read_csv('./Final_Charge_Zeroed_Example.cif.txt', header=None)[0].values

np.testing.assert_allclose(out['val_total'], other_total)
np.testing.assert_allclose(out['val_zero'], other_zero, rtol=1e-6, atol=1e-5)
np.testing.assert_allclose(out['val_shift'], other_shift, rtol=1e-6, atol=1e-5)